In [72]:
#Purpose: explore my network
import uuid
import networkx as nx
import numpy as np

In [73]:
class Wire:
    def __init__(self, router1, router2):
        # connection between two routers
        self.router1 = router1
        self.router2 = router2
        # need a tuple to jump on graph
        self.jump = (router1.id, router2.id)
        # list of packets on wire
        # acting as input buffer
        self.packets = []

    def __eq__(self, other):
        if not isinstance(self, other):
            return False

        return (self.router1 is other.router1) and (self.router2 is other.router2)
    
    def __repr__(self):
        return f"{self.router1.id},{self.router2.id}\n"

    def find_packet(self, packet):
        for p in self.packets:
            if p == packet:
                return p

        return None

    # Queue behaviour
    def insert_packet(self, packet):
        self.packets.append(packet)

    def remove_packet(self, packet, dst):
        p = self.find_packet(packet)

        if dst is self.router1:
            self.hop(p, self.router1)
        elif dst is self.router2:
            self.hop(p, self.router2)

    def hop(self, packet, dst):
        # if we successfully hopped to router, remove packet
        if packet and dst.insert_packet(packet):
            packet.push_to_router(dst)
            self.packets.remove(packet)
            return self.dst

        # operation failure, dst router congested
        return -1


In [74]:

class Router:
    def __init__(self, id, connections, kind, graph):
        # node on graph
        self.id = id
        self.network = graph
        # avoiding using type, better name?
        self.kind = kind

        # edges connected to this node
        # list of wires connected to routers
        self.connections = []

        # how many actions are possible in router
        if connections:
            self.actions = len(connections)

        # depending on type, set buffer size
        if kind == 'T':
            self.buffer_size = 100  # change this to hold more packets
        elif kind == 'M':
            self.buffer_size = 25  # change this to hold more packets
        elif kind == 'C' or kind == 'CP':
            self.buffer_size = 10

        self.buffer = []  # Queue

    def __eq__(self, other):
        if not isinstance(self, other):
            return False

        return self.id == other.id

    def __repr__(self):
        return f"id: {self.id}, type: {self.kind}\n connections: {self.connections}\n"

    # for visualization we need to know
    # if the router is full, active, and inactive
    # full = red; active = yellow; inactive = grey
    def is_full(self):
        return len(self.buffer) >= self.buffer_size

    def is_active(self):
        return len(self.buffer) != 0

    # can only add and find connections
    def has_connection(self, dst):
        for connection in self.connections:
            if connection == dst:
                return connection

        return None

    def add_connections(self, wires):
        for wire in wires:
            if self is wire.router1 or self is wire.router2:
                self.connections.append(wire)

    # no need for queue, just find and remove
    # todo: if this is the packet dst router, call network to remove packet from transmission
    def insert_packet(self, packet):
        if not self.is_full():
            # enqueue packet
            self.buffer.append(packet)
            return True

        return False

    # push to wire
    def remove_packet(self, packet, dst):
        # if there is a connection to the destination
        # and the buffer is not empty
        # and the packet exists
        # move packet to wire
        wire = self.has_connection(dst)
        packet_index = self.find_packet(packet)
        if wire and self.buffer and packet_index:
            # dequeue packet
            self.buffer[packet_index].push_to_wire(wire, dst)
            self.buffer.remove(packet)

    def find_packet(self, packet):
        for index, p in enumerate(self.buffer):
            if p is packet:
                return index
        return None

    # cant generate packets here, must generate from network and insert to routers
    # centralized controller controls packet path


In [75]:
class Packet:
    def __init__(self, src, dst, graph, path=[]):
        self.id = str(uuid.uuid4())
        self.curr = src
        self.next_hop = None
        self.src = src
        self.dst = dst
        self.path = path
        self.graph = graph

    # add an equal method to find
    def __eq__(self, other):
        if not isinstance(self, other):
            return False

        return self.id is other.id

    def __repr__(self):
        return f"id: {self.id},\n src: {self.curr}, dst: {self.dst}, curr: {self.curr}"


    def on_wire(self):
        return isinstance(self.curr, Wire)

    def on_router(self):
        return isinstance(self.curr, Router)

    def push_to_wire(self, wire, dst):
        if isinstance(self.curr, Router):
            self.curr = wire
            self.next_hop = dst

    def push_to_router(self, router):
        if isinstance(self.curr, Wire):
            self.curr = router
            self.update_path()
            self.next_hop = self.find_next_hop()

    def find_next_hop(self):
        # we are at destination
        if len(self.path) < 2:
            return -1
        # shortest path
        return self.path[1]

    def complete(self):
        return self.curr is self.dst

    def update_path(self):
        self.path = nx.shortest_path(self.graph, self.curr, self.dst)


In [76]:
G = nx.random_internet_as_graph(n=25)
nx.set_node_attributes(G, {})

all_nodes_data = np.array(G.nodes.data())

all_connections = np.array(G.edges())

#create router for each node
#create a wire for each connection
# then build the network

#all the routers and wires
routers = []
wires = []

#the routers that can generate new packets
customer_routers = []
customer_buffer_size = 10

#total amount of packets
packets = []

#extract data from nodes and edges and map
for node in all_nodes_data:
  id = node[0]
  kind = node[1]['type']
  r = Router(id=id,connections=[], kind=kind, graph=G)
  routers.append(r)


  if kind is 'C' or kind is 'CP':
    customer_routers.append(r)

for connection in all_connections:
  router_id_1 = connection[0]
  router_id_2 = connection[1]
  wire = Wire(routers[router_id_1], routers[router_id_2])
  wires.append(wire)

#building the network
for router in routers:
    router.add_connections(wires)

def generate_packets(amount=1):
  
  if amount > customer_buffer_size:
      raise Exception(f"cannot generate more packets than buffer size: {customer_buffer_size}")
  else:
    for i in range(amount):
      src = np.random.choice(customer_routers)
      dst = np.random.choice(customer_routers)
      # dst cannot be the same src
      # you cannot send packets to yourself
      while src is dst:
        dst = np.random.choice(customer_routers)
      else:
        p = Packet(src,dst,G)
        packets.append(p)

generate_packets()
print(packets)



[id: 8a9616a0-51d0-4828-84cd-dd4d5ff8f791,
 src: id: 9, type: C
 connections: [7,9
]
, dst: id: 16, type: C
 connections: [2,16
]
, curr: id: 9, type: C
 connections: [7,9
]
]


In [77]:
src = np.random.choice(customer_routers)
dst = np.random.choice(customer_routers)

p = Packet(src,dst,G)
packets.append(p)
packets

[id: 8a9616a0-51d0-4828-84cd-dd4d5ff8f791,
  src: id: 9, type: C
  connections: [7,9
 ]
 , dst: id: 16, type: C
  connections: [2,16
 ]
 , curr: id: 9, type: C
  connections: [7,9
 ], id: 8eb9aaba-b62a-4209-8ee2-3785a3b9a064,
  src: id: 16, type: C
  connections: [2,16
 ]
 , dst: id: 23, type: C
  connections: [3,23
 ]
 , curr: id: 16, type: C
  connections: [2,16
 ]]

In [78]:
def generate_packets(amount=1):
  
  if amount > customer_buffer_size:
      raise Exception(f"cannot generate more packets than buffer size: {customer_buffer_size}")
  else:
    for i in range(amount):
      src = np.random.choice(customer_routers)
      dst = np.random.choice(customer_routers)
      # dst cannot be the same src
      # you cannot send packets to yourself
      while src is dst:
        dst = np.random.choice(customer_routers)
      else:
        p = Packet(src,dst,G)
        packets.append(p)

generate_packets()
print(packets)



[id: 8a9616a0-51d0-4828-84cd-dd4d5ff8f791,
 src: id: 9, type: C
 connections: [7,9
]
, dst: id: 16, type: C
 connections: [2,16
]
, curr: id: 9, type: C
 connections: [7,9
]
, id: 8eb9aaba-b62a-4209-8ee2-3785a3b9a064,
 src: id: 16, type: C
 connections: [2,16
]
, dst: id: 23, type: C
 connections: [3,23
]
, curr: id: 16, type: C
 connections: [2,16
]
, id: 43d85661-62c2-4300-826e-3e6db7a2f1c9,
 src: id: 17, type: C
 connections: [2,17
]
, dst: id: 9, type: C
 connections: [7,9
]
, curr: id: 17, type: C
 connections: [2,17
]
]


In [79]:
np.random.choice(customer_routers)

id: 10, type: C
 connections: [4,10
]